### https://huggingface.co/transformers has pipelines, pretrained models, tokenizers etc for several tasks. 

In [1]:
import torch
from transformers import pipeline

### Sentiment analysis: recall we used it in our introductory lecture

In [2]:
classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

In [3]:
result = classifier("I hate washing dishes")[0]
print("Sentiment: {} Score: {}".format(result['label'],result['score']))

result = classifier("I love visiting Paris")[0]
print("Sentiment: {} Score: {}".format(result['label'],result['score']))

Sentiment: NEGATIVE Score: 0.9992689490318298
Sentiment: POSITIVE Score: 0.999553382396698


### You can also fine tune any pretrained model. For instance, let's fine tune a model for sentiment analysis

### Load a dataset for fine-tuning. You can get imdbs.csv from
### https://drive.google.com/uc?id=11_M4ootuT7I1G0RlihcC0cA3Elqotlc-

In [4]:
import datasets
from datasets import load_dataset

dataset = load_dataset('csv', data_files='./imdbs.csv', split='train')

Using custom data configuration default-4281c69669ee3403
Found cached dataset csv (/home/afalcao/.cache/huggingface/datasets/csv/default-4281c69669ee3403/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


In [5]:
# split dataset into train and test

dataset   = dataset.train_test_split(test_size=0.1)
train_set = dataset['train']
test_set  = dataset['test']

### Load the tokenizer and preprocess the training and test sets with the tokenizer -- it already converts tokens into ids and sets attention masks

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# preprocess the dataset 

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_set = train_set.map(tokenize_function, batched=True)
test_set  = test_set.map(tokenize_function, batched=True) 

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

### Load the model for sequence classification

In [7]:
from transformers import AutoModelForSequenceClassification

checkpoint = "bert-base-cased"

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

### Since we want to report the accuracy of the model, we can add the following function.

In [8]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

/tmp/ipykernel_16139/3043712826.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


### Now set training parameters and arguments, and train the model

In [9]:
from transformers import TrainingArguments, Trainer

In [10]:
# set training parameters and arguments

batch_size = 8
epochs     = 20
warmup_steps = 100
weight_decay = 0.01

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    evaluation_strategy="epoch",
    logging_dir='./logs',    
)

In [11]:
# define Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    compute_metrics=compute_metrics
)

In [12]:
# Train

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/afalcao/miniconda3/envs/mo434/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 90
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 240
  Number of trainable parameters = 108311810


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.826647,0.300000
2,No log,0.669995,0.700000
3,No log,0.598046,0.800000
4,No log,0.601246,0.700000
5,No log,0.480498,0.800000
6,No log,0.515017,0.700000
7,No log,0.285862,0.900000
8,No log,0.083808,1.000000
9,No log,1.356995,0.800000
10,No log,0.012094,1.000000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 8
The following columns 

TrainOutput(global_step=240, training_loss=0.2421143372853597, metrics={'train_runtime': 87.8693, 'train_samples_per_second': 20.485, 'train_steps_per_second': 2.731, 'total_flos': 473599899648000.0, 'train_loss': 0.2421143372853597, 'epoch': 20.0})

In [13]:
# evaluate 

trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10
  Batch size = 8


{'eval_loss': 0.00563486572355032,
 'eval_accuracy': 1.0,
 'eval_runtime': 0.1609,
 'eval_samples_per_second': 62.164,
 'eval_steps_per_second': 12.433,
 'epoch': 20.0}

In [14]:
# test

inputs  = tokenizer('High tech companies are growing up', return_tensors="pt").to(device=0)
labels  = torch.tensor([1]).unsqueeze(0).to(device=0)
outputs = model(**inputs, labels=labels)
loss    = outputs.loss
logits  = outputs.logits
answer  = torch.argmax(logits)
if (answer == 0):
    print("Sentiment: NEGATIVE")
else:
    print("Sentiment: POSITIVE")

Sentiment: POSITIVE


### Extractive Question Answering: the task of extracting an answer from a text given a question

In [15]:
question_answerer = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
loading configuration file config.json from cache at /home/afalcao/.cache/huggingface/hub/models--distilbert-base-cased-distilled-squad/snapshots/a8440a9ab1b3d7a603df3349516078434abade1e/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased-distilled-squad",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": true,
  "tie_weights_": true,
  "tran

In [16]:
context = "The immune system is a system of many biological structures and processes \
within an organism that protects against diseases. To function properly the immune system \
must detect a wide variety of agents, called pathogens."

result = question_answerer(question="What are pathogens?", context=context)
print("Answer: {}".format(result['answer']))
print("Score: {}".format(round(result['score'], 4)))
print("Start: {} End: {}".format(result['start'],result['end']))

result = question_answerer(question="How does the immune system work?", context=context)
print("Answer: {}".format(result['answer']))
print("Score: {}".format(round(result['score'], 4)))
print("Start: {} End: {}".format(result['start'],result['end']))

result = question_answerer(question="What is the immune system?", context=context)
print("Answer: {}".format(result['answer']))
print("Score: {}".format(round(result['score'], 4)))
print("Start: {} End: {}".format(result['start'],result['end']))

Answer: a wide variety of agents
Score: 0.4355
Start: 176 End: 200
Answer: must detect a wide variety of agents
Score: 0.058
Start: 164 End: 200
Answer: a system of many biological structures and processes within an organism that protects against diseases
Score: 0.4359
Start: 21 End: 123


### You may also use pretrained models already fine-tuned in some dataset (e.g., SQUAD -- Stanford Question-Answering Dataset).

In [17]:
from transformers import BertForQuestionAnswering, BertTokenizer
import torch

In [18]:
# load the fine-tuned model

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

loading configuration file config.json from cache at /home/afalcao/.cache/huggingface/hub/models--bert-large-uncased-whole-word-masking-finetuned-squad/snapshots/242d9dbb66bb5033025196d5678907307f8fb098/config.json
Model config BertConfig {
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /home/afalcao/.cache/huggingface/hub/models--bert-large-uncased-whole-word-masking-finetuned-squad/snapshots/242d9dbb66bb5033025196d5678907307f8fb098/pytorch_model.bin
All model checkpoint weights were used when initializing BertForQuestionAnswering.

All the weights of BertForQuestionAnswering were initialized from the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForQuestionAnswering for predictions without further training.


In [19]:
# load its fine-tuned tokenizer

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

loading file vocab.txt from cache at /home/afalcao/.cache/huggingface/hub/models--bert-large-uncased-whole-word-masking-finetuned-squad/snapshots/242d9dbb66bb5033025196d5678907307f8fb098/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/afalcao/.cache/huggingface/hub/models--bert-large-uncased-whole-word-masking-finetuned-squad/snapshots/242d9dbb66bb5033025196d5678907307f8fb098/tokenizer_config.json
loading configuration file config.json from cache at /home/afalcao/.cache/huggingface/hub/models--bert-large-uncased-whole-word-masking-finetuned-squad/snapshots/242d9dbb66bb5033025196d5678907307f8fb098/config.json
Model config BertConfig {
  "_name_or_path": "bert-large-uncased-whole-word-masking-finetuned-squad",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden

In [20]:
# add BERT tags to process question and context

question = "[CLS]" + "What are pathogens?" + "[SEP]"
context  = context + "[SEP]"

In [21]:
# get their tokens, combine and convert them into input_ids
question_tokens = tokenizer.tokenize(question)
context_tokens  = tokenizer.tokenize(context)
all_tokens      = question_tokens + context_tokens
input_ids       = tokenizer.convert_tokens_to_ids(all_tokens)

In [22]:
# define segment_ids with zeroes for question tokens and ones for context tokens

segment_ids = [0] * len(question_tokens)
segment_ids = segment_ids + [1] * len(context_tokens)

In [23]:
# convert input and segment ids to tensors and feed them into the model 
# to obtain the start and end scores

input_ids              = torch.tensor([input_ids])
segment_ids            = torch.tensor([segment_ids])

In [24]:
result = model(input_ids, token_type_ids = segment_ids)

In [25]:
#tokens with highest start and end scores
answer_start = torch.argmax(result.start_logits)
answer_end   = torch.argmax(result.end_logits)
if answer_end >= answer_start:
    answer = " ".join(all_tokens[answer_start:answer_end+1])
    print("\nQuestion:{}".format(question[5:-5]))
    print("\nAnswer: {}.".format(answer))
else:
    print("I could not find an answer to your question.")


Question:What are pathogens?

Answer: agents.


### Text Generation

In [27]:
text_generator = pipeline("text-generation")

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

loading configuration file config.json from cache at /home/afalcao/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transform

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /home/afalcao/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/pytorch_model.bin
All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /home/afalcao/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_rang

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

loading file vocab.json from cache at /home/afalcao/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/vocab.json
loading file merges.txt from cache at /home/afalcao/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/merges.txt
loading file tokenizer.json from cache at /home/afalcao/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /home/afalcao/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_toke

In [28]:
text = text_generator("It is a strong idea", max_length=35, do_sample=False)
print(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'It is a strong idea that the government should be able to provide a fair and transparent process for the public to make their own decisions about the future of their lives," he said'}]


### Named Entity Recognition

In [29]:
ner_pipe = pipeline("ner")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

loading configuration file config.json from cache at /home/afalcao/.cache/huggingface/hub/models--dbmdz--bert-large-cased-finetuned-conll03-english/snapshots/f2482bf01f5da0f0eb8e183ffd8cc3885aa90b14/config.json
Model config BertConfig {
  "_name_or_path": "dbmdz/bert-large-cased-finetuned-conll03-english",
  "_num_labels": 9,
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "O",
    "1": "B-MISC",
    "2": "I-MISC",
    "3": "B-PER",
    "4": "I-PER",
    "5": "B-ORG",
    "6": "I-ORG",
    "7": "B-LOC",
    "8": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "B-LOC": 7,
    "B-MISC": 1,
    "B-ORG": 5,
    "B-PER": 3,
    "I-LOC": 8,
    "I-MISC": 2,
    "I-ORG": 6,
    "I-PER": 4,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_posit

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

loading configuration file config.json from cache at /home/afalcao/.cache/huggingface/hub/models--dbmdz--bert-large-cased-finetuned-conll03-english/snapshots/f2482bf01f5da0f0eb8e183ffd8cc3885aa90b14/config.json
Model config BertConfig {
  "_num_labels": 9,
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "O",
    "1": "B-MISC",
    "2": "I-MISC",
    "3": "B-PER",
    "4": "I-PER",
    "5": "B-ORG",
    "6": "I-ORG",
    "7": "B-LOC",
    "8": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "B-LOC": 7,
    "B-MISC": 1,
    "B-ORG": 5,
    "B-PER": 3,
    "I-LOC": 8,
    "I-MISC": 2,
    "I-ORG": 6,
    "I-PER": 4,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 

...

loading weights file pytorch_model.bin from cache at /home/afalcao/.cache/huggingface/hub/models--dbmdz--bert-large-cased-finetuned-conll03-english/snapshots/f2482bf01f5da0f0eb8e183ffd8cc3885aa90b14/pytorch_model.bin
All model checkpoint weights were used when initializing BertForTokenClassification.

All the weights of BertForTokenClassification were initialized from the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForTokenClassification for predictions without further training.


Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

loading configuration file config.json from cache at /home/afalcao/.cache/huggingface/hub/models--dbmdz--bert-large-cased-finetuned-conll03-english/snapshots/f2482bf01f5da0f0eb8e183ffd8cc3885aa90b14/config.json
Model config BertConfig {
  "_name_or_path": "dbmdz/bert-large-cased-finetuned-conll03-english",
  "_num_labels": 9,
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "O",
    "1": "B-MISC",
    "2": "I-MISC",
    "3": "B-PER",
    "4": "I-PER",
    "5": "B-ORG",
    "6": "I-ORG",
    "7": "B-LOC",
    "8": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "B-LOC": 7,
    "B-MISC": 1,
    "B-ORG": 5,
    "B-PER": 3,
    "I-LOC": 8,
    "I-MISC": 2,
    "I-ORG": 6,
    "I-PER": 4,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_posit

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

loading file vocab.txt from cache at /home/afalcao/.cache/huggingface/hub/models--dbmdz--bert-large-cased-finetuned-conll03-english/snapshots/f2482bf01f5da0f0eb8e183ffd8cc3885aa90b14/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/afalcao/.cache/huggingface/hub/models--dbmdz--bert-large-cased-finetuned-conll03-english/snapshots/f2482bf01f5da0f0eb8e183ffd8cc3885aa90b14/tokenizer_config.json
loading configuration file config.json from cache at /home/afalcao/.cache/huggingface/hub/models--dbmdz--bert-large-cased-finetuned-conll03-english/snapshots/f2482bf01f5da0f0eb8e183ffd8cc3885aa90b14/config.json
Model config BertConfig {
  "_name_or_path": "dbmdz/bert-large-cased-finetuned-conll03-english",
  "_num_labels": 9,
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifi

In [30]:
text = "IBM introduces Eagle in USA -- the first processor to surpass 100 qubits."
result = ner_pipe(text)
for entity in result:
    print(entity)

{'entity': 'I-ORG', 'score': 0.9989236, 'index': 1, 'word': 'IBM', 'start': 0, 'end': 3}
{'entity': 'I-MISC', 'score': 0.767113, 'index': 3, 'word': 'Eagle', 'start': 15, 'end': 20}
{'entity': 'I-LOC', 'score': 0.9996847, 'index': 5, 'word': 'USA', 'start': 24, 'end': 27}


### Text Summarization

In [31]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

loading configuration file config.json from cache at /home/afalcao/.cache/huggingface/hub/models--sshleifer--distilbart-cnn-12-6/snapshots/a4f8f3ea906ed274767e9906dbaede7531d660ff/config.json
Model config BartConfig {
  "_name_or_path": "sshleifer/distilbart-cnn-12-6",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bo

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

loading configuration file config.json from cache at /home/afalcao/.cache/huggingface/hub/models--sshleifer--distilbart-cnn-12-6/snapshots/a4f8f3ea906ed274767e9906dbaede7531d660ff/config.json
Model config BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradi

...

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

loading configuration file config.json from cache at /home/afalcao/.cache/huggingface/hub/models--sshleifer--distilbart-cnn-12-6/snapshots/a4f8f3ea906ed274767e9906dbaede7531d660ff/config.json
Model config BartConfig {
  "_name_or_path": "sshleifer/distilbart-cnn-12-6",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bo

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

loading file vocab.json from cache at /home/afalcao/.cache/huggingface/hub/models--sshleifer--distilbart-cnn-12-6/snapshots/a4f8f3ea906ed274767e9906dbaede7531d660ff/vocab.json
loading file merges.txt from cache at /home/afalcao/.cache/huggingface/hub/models--sshleifer--distilbart-cnn-12-6/snapshots/a4f8f3ea906ed274767e9906dbaede7531d660ff/merges.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/afalcao/.cache/huggingface/hub/models--sshleifer--distilbart-cnn-12-6/snapshots/a4f8f3ea906ed274767e9906dbaede7531d660ff/tokenizer_config.json
loading configuration file config.json from cache at /home/afalcao/.cache/huggingface/hub/models--sshleifer--distilbart-cnn-12-6/snapshots/a4f8f3ea906ed274767e9906dbaede7531d660ff/config.json
Model config BartConfig {
  "_name_or_path": "sshleifer/distilbart-cnn-12-6",
  "_num_labels": 3,
  "acti

In [32]:
text = "IBM has unveiled an advanced quantum processor that is part of an effort \
to build super-fast computers. These machines could revolutionise computing, harnessing \
the strange world of quantum physics to solve problems beyond reach for even the most \
advanced classical ones. But the hurdles in building practical, large-scale versions \
have kept quantum computers confined to the lab. The new chip has 127 qubits, \
twice as many as the previous IBM processor. Qubits (quantum bits) are the most basic \
units of information in a quantum computer. The company called its new Eagle processor \
a key milestone on the path towards practical quantum computation."

In [33]:
result = summarizer(text, max_length=50, min_length=30, do_sample=False)

In [34]:
print(result[0]['summary_text'])

 IBM has unveiled an advanced quantum processor that is part of an effort to build super-fast computers . These machines could revolutionise computing, harnessing the strange world of quantum physics to solve problems beyond reach for even the most advanced classical ones


### Translation: For example, from English to French

In [35]:
translator = pipeline("translation_en_to_fr")

No model was supplied, defaulted to t5-base and revision 686f1db (https://huggingface.co/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

loading configuration file config.json from cache at /home/afalcao/.cache/huggingface/hub/models--t5-base/snapshots/0db7e623bcaee2daf9b859a646637ea39bf016cd/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

loading configuration file config.json from cache at /home/afalcao/.cache/huggingface/hub/models--t5-base/snapshots/0db7e623bcaee2daf9b859a646637ea39bf016cd/config.json
Model config T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
  

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

ValueError: Could not load model t5-base with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSeq2SeqLM'>, <class 'transformers.models.t5.modeling_t5.T5ForConditionalGeneration'>).

In [ ]:
result = translator("The book is on the table.")
print(result[0]['translation_text'])